In [1]:
import os
import pandas as pd

# Import the required ArcGIS API for Python modules
import arcgis
from arcgis.gis import GIS
gis = arcgis.GIS()

In [8]:
# read in segments layer
sdfSegmentsWithFactors = pd.DataFrame.spatial.from_featureclass('../results/layers/Master_Segs_withFactors_20211119.shp')

#create map1 that centers on Salt Lake
map1 = gis.map('Salt Lake')

# add _df to map
sdfSegmentsWithFactors.spatial.plot(map_widget = map1)

#show map1
map1

MapView(layout=Layout(height='400px', width='100%'))

In [3]:
# delete CO_FIPS=0 ... mostly for transit segs that have since been corrected
sdfSegmentsWithFactors = sdfSegmentsWithFactors[sdfSegmentsWithFactors['CO_FIPS']>0].copy()

# create only two classes, freeway or arterial
sdfSegmentsWithFactors.loc[sdfSegmentsWithFactors['FC_GROUP'].str.contains('Freeway', na=False), 'FC_GROUP'] = 'Freeway'
sdfSegmentsWithFactors.loc[~sdfSegmentsWithFactors['FC_GROUP'].str.contains('Freeway', na=False), 'FC_GROUP'] = 'Arterial'

sdfSegmentsWithFactors

,FID,Id,SEGID,BMP,EMP,DISTANCE,FULLNAME,FNCLASS,FC_CODE,FC_GROUP,...,FAC_WIN,FAC_SPR,FAC_SUM,FAC_FAL,FAC_MAXMO,FAC_MAX,FACMANADJ,SUTRUCKS,CUTRUCKS,SHAPE
0,0,0,0006_000.0,0.000,0.665,0.666641,HWY 6,Arterial,3,Arterial,...,0.8495,1.0276,1.0914,1.0316,5,1.1065,0,0.2496,0.2324,"{""paths"": [[[236177.7000000002, 4327541.25], [..."
1,1,0,0006_000.7,0.665,16.022,15.369839,HWY 6,Arterial,3,Arterial,...,0.8495,1.0276,1.0914,1.0316,5,1.1065,0,0.2496,0.2324,"{""paths"": [[[237241.1799999997, 4327399.720000..."
2,2,0,0006_016.0,16.022,46.017,30.001961,HWY 6,Arterial,3,Arterial,...,0.8495,1.0276,1.0914,1.0316,5,1.1065,0,0.2496,0.2324,"{""paths"": [[[261403.90000000037, 4327045.4], [..."
3,3,0,0006_046.0,46.017,60.218,14.194306,HWY 6,Arterial,3,Arterial,...,0.8495,1.0276,1.0914,1.0316,5,1.1065,0,0.1751,0.3338,"{""paths"": [[[305380.16000000015, 4325741.85], ..."
4,4,0,0006_060.2,60.218,77.545,17.323237,HWY 6,Arterial,3,Arterial,...,0.8495,1.0276,1.0914,1.0316,5,1.1065,0,0.1751,0.3338,"{""paths"": [[[324618, 4337936.9], [324647.40000..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8558,8558,0,WFRC_8219,0.000,0.000,1.154153,,Arterial,50,Arterial,...,0.8994,1.0107,1.0575,1.0324,8,1.0724,0,0.1278,0.0879,"{""paths"": [[[419504.53110000025, 4554272.0693]..."
8559,8559,0,WFRC_8220,0.000,0.000,0.192948,AIRPORT RD,Arterial,4,Arterial,...,0.9370,1.0104,1.0283,1.0243,9,1.0434,0,0.1143,0.0577,"{""paths"": [[[415710.1200000001, 4495585.710000..."
8560,8560,0,WFRC_8221,0.000,0.000,0.442365,9400 S,Arterial,2,Arterial,...,0.9653,1.0117,1.0135,1.0095,5,1.0245,0,0.1097,0.0482,"{""paths"": [[[423366.5, 4492605.300000001], [42..."
8561,8561,0,WFRC_8222,0.000,0.000,0.442365,300 W,Arterial,5,Arterial,...,0.9653,1.0117,1.0135,1.0095,5,1.0245,0,0.1097,0.0482,"{""paths"": [[[423433.7000000002, 4493180.6], [4..."


In [4]:
# calculate 2019 VMT
sdfSegmentsWithFactors['VMT'] = sdfSegmentsWithFactors['DISTANCE'] * sdfSegmentsWithFactors['AADT2019']

# Saturday and Sunday
sdfSegmentsWithFactors['FAC_JAN_WEAVG_x_VMT'] = sdfSegmentsWithFactors['FAC_JAN'] * sdfSegmentsWithFactors['FAC_WEAVG'] * sdfSegmentsWithFactors['VMT']
sdfSegmentsWithFactors['FAC_FEB_WEAVG_x_VMT'] = sdfSegmentsWithFactors['FAC_FEB'] * sdfSegmentsWithFactors['FAC_WEAVG'] * sdfSegmentsWithFactors['VMT']
sdfSegmentsWithFactors['FAC_MAR_WEAVG_x_VMT'] = sdfSegmentsWithFactors['FAC_MAR'] * sdfSegmentsWithFactors['FAC_WEAVG'] * sdfSegmentsWithFactors['VMT']
sdfSegmentsWithFactors['FAC_APR_WEAVG_x_VMT'] = sdfSegmentsWithFactors['FAC_APR'] * sdfSegmentsWithFactors['FAC_WEAVG'] * sdfSegmentsWithFactors['VMT']
sdfSegmentsWithFactors['FAC_MAY_WEAVG_x_VMT'] = sdfSegmentsWithFactors['FAC_MAY'] * sdfSegmentsWithFactors['FAC_WEAVG'] * sdfSegmentsWithFactors['VMT']
sdfSegmentsWithFactors['FAC_JUN_WEAVG_x_VMT'] = sdfSegmentsWithFactors['FAC_JUN'] * sdfSegmentsWithFactors['FAC_WEAVG'] * sdfSegmentsWithFactors['VMT']
sdfSegmentsWithFactors['FAC_JUL_WEAVG_x_VMT'] = sdfSegmentsWithFactors['FAC_JUL'] * sdfSegmentsWithFactors['FAC_WEAVG'] * sdfSegmentsWithFactors['VMT']
sdfSegmentsWithFactors['FAC_AUG_WEAVG_x_VMT'] = sdfSegmentsWithFactors['FAC_AUG'] * sdfSegmentsWithFactors['FAC_WEAVG'] * sdfSegmentsWithFactors['VMT']
sdfSegmentsWithFactors['FAC_SEP_WEAVG_x_VMT'] = sdfSegmentsWithFactors['FAC_SEP'] * sdfSegmentsWithFactors['FAC_WEAVG'] * sdfSegmentsWithFactors['VMT']
sdfSegmentsWithFactors['FAC_OCT_WEAVG_x_VMT'] = sdfSegmentsWithFactors['FAC_OCT'] * sdfSegmentsWithFactors['FAC_WEAVG'] * sdfSegmentsWithFactors['VMT']
sdfSegmentsWithFactors['FAC_NOV_WEAVG_x_VMT'] = sdfSegmentsWithFactors['FAC_NOV'] * sdfSegmentsWithFactors['FAC_WEAVG'] * sdfSegmentsWithFactors['VMT']
sdfSegmentsWithFactors['FAC_DEC_WEAVG_x_VMT'] = sdfSegmentsWithFactors['FAC_DEC'] * sdfSegmentsWithFactors['FAC_WEAVG'] * sdfSegmentsWithFactors['VMT']

sdfSegmentsWithFactors['FAC_JAN_WDAVG_x_VMT'] = sdfSegmentsWithFactors['FAC_JAN'] * sdfSegmentsWithFactors['FAC_WDAVG'] * sdfSegmentsWithFactors['VMT']
sdfSegmentsWithFactors['FAC_FEB_WDAVG_x_VMT'] = sdfSegmentsWithFactors['FAC_FEB'] * sdfSegmentsWithFactors['FAC_WDAVG'] * sdfSegmentsWithFactors['VMT']
sdfSegmentsWithFactors['FAC_MAR_WDAVG_x_VMT'] = sdfSegmentsWithFactors['FAC_MAR'] * sdfSegmentsWithFactors['FAC_WDAVG'] * sdfSegmentsWithFactors['VMT']
sdfSegmentsWithFactors['FAC_APR_WDAVG_x_VMT'] = sdfSegmentsWithFactors['FAC_APR'] * sdfSegmentsWithFactors['FAC_WDAVG'] * sdfSegmentsWithFactors['VMT']
sdfSegmentsWithFactors['FAC_MAY_WDAVG_x_VMT'] = sdfSegmentsWithFactors['FAC_MAY'] * sdfSegmentsWithFactors['FAC_WDAVG'] * sdfSegmentsWithFactors['VMT']
sdfSegmentsWithFactors['FAC_JUN_WDAVG_x_VMT'] = sdfSegmentsWithFactors['FAC_JUN'] * sdfSegmentsWithFactors['FAC_WDAVG'] * sdfSegmentsWithFactors['VMT']
sdfSegmentsWithFactors['FAC_JUL_WDAVG_x_VMT'] = sdfSegmentsWithFactors['FAC_JUL'] * sdfSegmentsWithFactors['FAC_WDAVG'] * sdfSegmentsWithFactors['VMT']
sdfSegmentsWithFactors['FAC_AUG_WDAVG_x_VMT'] = sdfSegmentsWithFactors['FAC_AUG'] * sdfSegmentsWithFactors['FAC_WDAVG'] * sdfSegmentsWithFactors['VMT']
sdfSegmentsWithFactors['FAC_SEP_WDAVG_x_VMT'] = sdfSegmentsWithFactors['FAC_SEP'] * sdfSegmentsWithFactors['FAC_WDAVG'] * sdfSegmentsWithFactors['VMT']
sdfSegmentsWithFactors['FAC_OCT_WDAVG_x_VMT'] = sdfSegmentsWithFactors['FAC_OCT'] * sdfSegmentsWithFactors['FAC_WDAVG'] * sdfSegmentsWithFactors['VMT']
sdfSegmentsWithFactors['FAC_NOV_WDAVG_x_VMT'] = sdfSegmentsWithFactors['FAC_NOV'] * sdfSegmentsWithFactors['FAC_WDAVG'] * sdfSegmentsWithFactors['VMT']
sdfSegmentsWithFactors['FAC_DEC_WDAVG_x_VMT'] = sdfSegmentsWithFactors['FAC_DEC'] * sdfSegmentsWithFactors['FAC_WDAVG'] * sdfSegmentsWithFactors['VMT']

display(sdfSegmentsWithFactors)

,FID,Id,SEGID,BMP,EMP,DISTANCE,FULLNAME,FNCLASS,FC_CODE,FC_GROUP,...,FAC_MAR_WDAVG_x_VMT,FAC_APR_WDAVG_x_VMT,FAC_MAY_WDAVG_x_VMT,FAC_JUN_WDAVG_x_VMT,FAC_JUL_WDAVG_x_VMT,FAC_AUG_WDAVG_x_VMT,FAC_SEP_WDAVG_x_VMT,FAC_OCT_WDAVG_x_VMT,FAC_NOV_WDAVG_x_VMT,FAC_DEC_WDAVG_x_VMT
0,0,0,0006_000.0,0.000,0.665,0.666641,HWY 6,Arterial,3,Arterial,...,255.786706,282.192954,301.221786,300.731774,293.898817,296.648333,292.701007,290.849848,258.917344,240.160741
1,1,0,0006_000.7,0.665,16.022,15.369839,HWY 6,Arterial,3,Arterial,...,5897.331278,6506.144745,6944.867028,6933.569458,6776.031128,6839.423046,6748.414847,6705.735140,5969.510194,5537.064340
2,2,0,0006_016.0,16.022,46.017,30.001961,HWY 6,Arterial,3,Arterial,...,11511.604018,12700.009286,13556.396175,13534.343294,13226.828116,13350.569283,13172.921073,13089.610188,11652.497426,10808.362138
3,3,0,0006_046.0,46.017,60.218,14.194306,HWY 6,Arterial,3,Arterial,...,4881.971215,5385.963560,5749.149797,5739.797361,5609.382847,5661.860401,5586.521339,5551.189917,4941.722885,4583.732445
4,4,0,0006_060.2,60.218,77.545,17.323237,HWY 6,Arterial,3,Arterial,...,5958.131701,6573.221926,7016.467356,7005.053310,6845.890788,6909.936265,6817.989789,6774.870061,6031.054769,5594.150476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8558,8558,0,WFRC_8219,0.000,0.000,1.154153,,Arterial,50,Arterial,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8559,8559,0,WFRC_8220,0.000,0.000,0.192948,AIRPORT RD,Arterial,4,Arterial,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8560,8560,0,WFRC_8221,0.000,0.000,0.442365,9400 S,Arterial,2,Arterial,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8561,8561,0,WFRC_8222,0.000,0.000,0.442365,300 W,Arterial,5,Arterial,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [5]:
# aggregate to county, month, 

sdfCoAgg = sdfSegmentsWithFactors.groupby(['CO_FIPS','FC_GROUP'], as_index=False).agg(
    
    SUM_FAC_JAN_WEAVG_x_VMT=('FAC_JAN_WEAVG_x_VMT', 'sum'),
    SUM_FAC_FEB_WEAVG_x_VMT=('FAC_FEB_WEAVG_x_VMT', 'sum'),
    SUM_FAC_MAR_WEAVG_x_VMT=('FAC_MAR_WEAVG_x_VMT', 'sum'),
    SUM_FAC_APR_WEAVG_x_VMT=('FAC_APR_WEAVG_x_VMT', 'sum'),
    SUM_FAC_MAY_WEAVG_x_VMT=('FAC_MAY_WEAVG_x_VMT', 'sum'),
    SUM_FAC_JUN_WEAVG_x_VMT=('FAC_JUN_WEAVG_x_VMT', 'sum'),
    SUM_FAC_JUL_WEAVG_x_VMT=('FAC_JUL_WEAVG_x_VMT', 'sum'),
    SUM_FAC_AUG_WEAVG_x_VMT=('FAC_AUG_WEAVG_x_VMT', 'sum'),
    SUM_FAC_SEP_WEAVG_x_VMT=('FAC_SEP_WEAVG_x_VMT', 'sum'),
    SUM_FAC_OCT_WEAVG_x_VMT=('FAC_OCT_WEAVG_x_VMT', 'sum'),
    SUM_FAC_NOV_WEAVG_x_VMT=('FAC_NOV_WEAVG_x_VMT', 'sum'),
    SUM_FAC_DEC_WEAVG_x_VMT=('FAC_DEC_WEAVG_x_VMT', 'sum'),

    SUM_FAC_JAN_WDAVG_x_VMT=('FAC_JAN_WDAVG_x_VMT', 'sum'),
    SUM_FAC_FEB_WDAVG_x_VMT=('FAC_FEB_WDAVG_x_VMT', 'sum'),
    SUM_FAC_MAR_WDAVG_x_VMT=('FAC_MAR_WDAVG_x_VMT', 'sum'),
    SUM_FAC_APR_WDAVG_x_VMT=('FAC_APR_WDAVG_x_VMT', 'sum'),
    SUM_FAC_MAY_WDAVG_x_VMT=('FAC_MAY_WDAVG_x_VMT', 'sum'),
    SUM_FAC_JUN_WDAVG_x_VMT=('FAC_JUN_WDAVG_x_VMT', 'sum'),
    SUM_FAC_JUL_WDAVG_x_VMT=('FAC_JUL_WDAVG_x_VMT', 'sum'),
    SUM_FAC_AUG_WDAVG_x_VMT=('FAC_AUG_WDAVG_x_VMT', 'sum'),
    SUM_FAC_SEP_WDAVG_x_VMT=('FAC_SEP_WDAVG_x_VMT', 'sum'),
    SUM_FAC_OCT_WDAVG_x_VMT=('FAC_OCT_WDAVG_x_VMT', 'sum'),
    SUM_FAC_NOV_WDAVG_x_VMT=('FAC_NOV_WDAVG_x_VMT', 'sum'),
    SUM_FAC_DEC_WDAVG_x_VMT=('FAC_DEC_WDAVG_x_VMT', 'sum'),

    SUM_VMT                =('VMT'                , 'sum')
    
)

sdfCoAgg

,CO_FIPS,FC_GROUP,SUM_FAC_JAN_WEAVG_x_VMT,SUM_FAC_FEB_WEAVG_x_VMT,SUM_FAC_MAR_WEAVG_x_VMT,SUM_FAC_APR_WEAVG_x_VMT,SUM_FAC_MAY_WEAVG_x_VMT,SUM_FAC_JUN_WEAVG_x_VMT,SUM_FAC_JUL_WEAVG_x_VMT,SUM_FAC_AUG_WEAVG_x_VMT,...,SUM_FAC_APR_WDAVG_x_VMT,SUM_FAC_MAY_WDAVG_x_VMT,SUM_FAC_JUN_WDAVG_x_VMT,SUM_FAC_JUL_WDAVG_x_VMT,SUM_FAC_AUG_WDAVG_x_VMT,SUM_FAC_SEP_WDAVG_x_VMT,SUM_FAC_OCT_WDAVG_x_VMT,SUM_FAC_NOV_WDAVG_x_VMT,SUM_FAC_DEC_WDAVG_x_VMT,SUM_VMT
0,1,Arterial,1.269498e+05,1.326417e+05,1.471867e+05,1.613266e+05,1.723691e+05,1.735172e+05,1.700001e+05,1.710695e+05,...,1.729207e+05,1.844135e+05,1.855925e+05,1.817467e+05,1.832650e+05,1.808938e+05,1.786231e+05,1.599232e+05,1.487277e+05,1.672498e+05
1,1,Freeway,5.599886e+05,6.221848e+05,7.785317e+05,7.952911e+05,7.600591e+05,8.385677e+05,8.708203e+05,8.009521e+05,...,6.476545e+05,6.189629e+05,6.828973e+05,7.091626e+05,6.522646e+05,5.931222e+05,6.291536e+05,6.087722e+05,5.403490e+05,6.702635e+05
2,3,Arterial,7.405140e+05,7.758786e+05,8.396923e+05,8.829062e+05,9.338376e+05,9.881212e+05,9.934759e+05,9.841920e+05,...,1.021050e+06,1.075254e+06,1.129436e+06,1.131130e+06,1.127575e+06,1.086935e+06,1.068120e+06,9.774834e+05,9.393775e+05,9.990211e+05
3,3,Freeway,1.404639e+06,1.469787e+06,1.721258e+06,1.797758e+06,1.877355e+06,2.106342e+06,2.250109e+06,2.137965e+06,...,1.657276e+06,1.729620e+06,1.939448e+06,2.069287e+06,1.967137e+06,1.758836e+06,1.723033e+06,1.614948e+06,1.507431e+06,1.766754e+06
4,5,Arterial,1.357075e+06,1.418102e+06,1.501452e+06,1.576204e+06,1.695799e+06,1.921367e+06,2.116257e+06,2.023484e+06,...,1.954991e+06,2.062108e+06,2.230372e+06,2.346048e+06,2.303430e+06,2.119215e+06,2.033794e+06,1.888315e+06,1.833692e+06,1.935014e+06
5,7,Arterial,4.778471e+05,5.017963e+05,5.515608e+05,5.846049e+05,6.261150e+05,6.767239e+05,6.833616e+05,6.654748e+05,...,6.429413e+05,6.851630e+05,7.331739e+05,7.369661e+05,7.232304e+05,6.885725e+05,6.736546e+05,6.079265e+05,5.807192e+05,6.411137e+05
6,7,Freeway,5.232282e+04,5.469897e+04,5.853611e+04,6.103244e+04,6.321518e+04,6.379851e+04,6.219112e+04,6.427941e+04,...,8.771397e+04,9.085406e+04,9.169272e+04,8.938242e+04,9.238434e+04,9.162011e+04,9.051721e+04,8.580283e+04,8.178171e+04,7.934665e+04
7,9,Arterial,4.825664e+04,5.016943e+04,5.796075e+04,6.890421e+04,8.650527e+04,1.184427e+05,1.262669e+05,1.110027e+05,...,6.375854e+04,7.967412e+04,1.083083e+05,1.152763e+05,1.016085e+05,8.462188e+04,7.035829e+04,5.298313e+04,4.680212e+04,7.524209e+04
8,11,Arterial,2.043367e+06,2.136448e+06,2.228118e+06,2.289272e+06,2.331481e+06,2.358637e+06,2.305619e+06,2.362810e+06,...,3.267166e+06,3.323121e+06,3.353106e+06,3.274810e+06,3.361062e+06,3.357431e+06,3.318103e+06,3.195934e+06,3.112839e+06,2.955733e+06
9,11,Freeway,3.173064e+06,3.325912e+06,3.504087e+06,3.611243e+06,3.628625e+06,3.719124e+06,3.685330e+06,3.713496e+06,...,4.865395e+06,4.888495e+06,5.006866e+06,4.953278e+06,4.996903e+06,4.889969e+06,4.900747e+06,4.751729e+06,4.703068e+06,4.449560e+06


In [6]:
sdfCoAgg['FAC_JAN_WEAVG'] = sdfCoAgg['SUM_FAC_JAN_WEAVG_x_VMT'] / sdfCoAgg['SUM_VMT']
sdfCoAgg['FAC_FEB_WEAVG'] = sdfCoAgg['SUM_FAC_FEB_WEAVG_x_VMT'] / sdfCoAgg['SUM_VMT']
sdfCoAgg['FAC_MAR_WEAVG'] = sdfCoAgg['SUM_FAC_MAR_WEAVG_x_VMT'] / sdfCoAgg['SUM_VMT']
sdfCoAgg['FAC_APR_WEAVG'] = sdfCoAgg['SUM_FAC_APR_WEAVG_x_VMT'] / sdfCoAgg['SUM_VMT']
sdfCoAgg['FAC_MAY_WEAVG'] = sdfCoAgg['SUM_FAC_MAY_WEAVG_x_VMT'] / sdfCoAgg['SUM_VMT']
sdfCoAgg['FAC_JUN_WEAVG'] = sdfCoAgg['SUM_FAC_JUN_WEAVG_x_VMT'] / sdfCoAgg['SUM_VMT']
sdfCoAgg['FAC_JUL_WEAVG'] = sdfCoAgg['SUM_FAC_JUL_WEAVG_x_VMT'] / sdfCoAgg['SUM_VMT']
sdfCoAgg['FAC_AUG_WEAVG'] = sdfCoAgg['SUM_FAC_AUG_WEAVG_x_VMT'] / sdfCoAgg['SUM_VMT']
sdfCoAgg['FAC_SEP_WEAVG'] = sdfCoAgg['SUM_FAC_SEP_WEAVG_x_VMT'] / sdfCoAgg['SUM_VMT']
sdfCoAgg['FAC_OCT_WEAVG'] = sdfCoAgg['SUM_FAC_OCT_WEAVG_x_VMT'] / sdfCoAgg['SUM_VMT']
sdfCoAgg['FAC_NOV_WEAVG'] = sdfCoAgg['SUM_FAC_NOV_WEAVG_x_VMT'] / sdfCoAgg['SUM_VMT']
sdfCoAgg['FAC_DEC_WEAVG'] = sdfCoAgg['SUM_FAC_DEC_WEAVG_x_VMT'] / sdfCoAgg['SUM_VMT']

sdfCoAgg['FAC_JAN_WDAVG'] = sdfCoAgg['SUM_FAC_JAN_WDAVG_x_VMT'] / sdfCoAgg['SUM_VMT']
sdfCoAgg['FAC_FEB_WDAVG'] = sdfCoAgg['SUM_FAC_FEB_WDAVG_x_VMT'] / sdfCoAgg['SUM_VMT']
sdfCoAgg['FAC_MAR_WDAVG'] = sdfCoAgg['SUM_FAC_MAR_WDAVG_x_VMT'] / sdfCoAgg['SUM_VMT']
sdfCoAgg['FAC_APR_WDAVG'] = sdfCoAgg['SUM_FAC_APR_WDAVG_x_VMT'] / sdfCoAgg['SUM_VMT']
sdfCoAgg['FAC_MAY_WDAVG'] = sdfCoAgg['SUM_FAC_MAY_WDAVG_x_VMT'] / sdfCoAgg['SUM_VMT']
sdfCoAgg['FAC_JUN_WDAVG'] = sdfCoAgg['SUM_FAC_JUN_WDAVG_x_VMT'] / sdfCoAgg['SUM_VMT']
sdfCoAgg['FAC_JUL_WDAVG'] = sdfCoAgg['SUM_FAC_JUL_WDAVG_x_VMT'] / sdfCoAgg['SUM_VMT']
sdfCoAgg['FAC_AUG_WDAVG'] = sdfCoAgg['SUM_FAC_AUG_WDAVG_x_VMT'] / sdfCoAgg['SUM_VMT']
sdfCoAgg['FAC_SEP_WDAVG'] = sdfCoAgg['SUM_FAC_SEP_WDAVG_x_VMT'] / sdfCoAgg['SUM_VMT']
sdfCoAgg['FAC_OCT_WDAVG'] = sdfCoAgg['SUM_FAC_OCT_WDAVG_x_VMT'] / sdfCoAgg['SUM_VMT']
sdfCoAgg['FAC_NOV_WDAVG'] = sdfCoAgg['SUM_FAC_NOV_WDAVG_x_VMT'] / sdfCoAgg['SUM_VMT']
sdfCoAgg['FAC_DEC_WDAVG'] = sdfCoAgg['SUM_FAC_DEC_WDAVG_x_VMT'] / sdfCoAgg['SUM_VMT']


dfCoAgg = sdfCoAgg[['CO_FIPS'      ,
                    'FC_GROUP'     ,
                    'FAC_JAN_WEAVG',
                    'FAC_FEB_WEAVG',
                    'FAC_MAR_WEAVG',
                    'FAC_APR_WEAVG',
                    'FAC_MAY_WEAVG',
                    'FAC_JUN_WEAVG',
                    'FAC_JUL_WEAVG',
                    'FAC_AUG_WEAVG',
                    'FAC_SEP_WEAVG',
                    'FAC_OCT_WEAVG',
                    'FAC_NOV_WEAVG',
                    'FAC_DEC_WEAVG',
                    'FAC_JAN_WDAVG',
                    'FAC_FEB_WDAVG',
                    'FAC_MAR_WDAVG',
                    'FAC_APR_WDAVG',
                    'FAC_MAY_WDAVG',
                    'FAC_JUN_WDAVG',
                    'FAC_JUL_WDAVG',
                    'FAC_AUG_WDAVG',
                    'FAC_SEP_WDAVG',
                    'FAC_OCT_WDAVG',
                    'FAC_NOV_WDAVG',
                    'FAC_DEC_WDAVG']]

display(dfCoAgg)

,CO_FIPS,FC_GROUP,FAC_JAN_WEAVG,FAC_FEB_WEAVG,FAC_MAR_WEAVG,FAC_APR_WEAVG,FAC_MAY_WEAVG,FAC_JUN_WEAVG,FAC_JUL_WEAVG,FAC_AUG_WEAVG,...,FAC_MAR_WDAVG,FAC_APR_WDAVG,FAC_MAY_WDAVG,FAC_JUN_WDAVG,FAC_JUL_WDAVG,FAC_AUG_WDAVG,FAC_SEP_WDAVG,FAC_OCT_WDAVG,FAC_NOV_WDAVG,FAC_DEC_WDAVG
0,1,Arterial,0.759043,0.793075,0.880041,0.964585,1.030609,1.037473,1.016444,1.022838,...,0.946098,1.033907,1.102623,1.109672,1.086678,1.095756,1.081579,1.068002,0.956193,0.889255
1,1,Freeway,0.835475,0.928269,1.161531,1.186535,1.133971,1.251102,1.299221,1.194981,...,0.945906,0.966269,0.923462,1.018849,1.058036,0.973146,0.884909,0.938666,0.908258,0.806174
2,3,Arterial,0.741240,0.776639,0.840515,0.883771,0.934753,0.989089,0.994449,0.985156,...,0.974207,1.022050,1.076307,1.130542,1.132238,1.128680,1.088000,1.069167,0.978441,0.940298
3,3,Freeway,0.795040,0.831914,0.974249,1.017549,1.062602,1.192210,1.273584,1.210110,...,0.898212,0.938034,0.978982,1.097747,1.171237,1.113419,0.995519,0.975253,0.914076,0.853221
4,5,Arterial,0.701326,0.732864,0.775939,0.814570,0.876376,0.992947,1.093665,1.045720,...,0.969272,1.010324,1.065681,1.152639,1.212419,1.190394,1.095193,1.051049,0.975866,0.947638
5,7,Arterial,0.745339,0.782695,0.860317,0.911858,0.976605,1.055544,1.065898,1.037998,...,0.948317,1.002851,1.068707,1.143594,1.149509,1.128084,1.074026,1.050757,0.948235,0.905797
6,7,Freeway,0.659421,0.689367,0.737726,0.769187,0.796696,0.804048,0.783790,0.810109,...,1.060211,1.105453,1.145027,1.155597,1.126480,1.164313,1.154682,1.140782,1.081367,1.030689
7,9,Arterial,0.641352,0.666773,0.770323,0.915767,1.149693,1.574155,1.678142,1.475275,...,0.714508,0.847379,1.058903,1.439464,1.532072,1.350421,1.124661,0.935092,0.704169,0.622020
8,11,Arterial,0.691323,0.722815,0.753829,0.774519,0.788799,0.797987,0.780050,0.799399,...,1.076594,1.105365,1.124297,1.134441,1.107952,1.137133,1.135905,1.122599,1.081266,1.053153
9,11,Freeway,0.713119,0.747470,0.787513,0.811596,0.815502,0.835841,0.828246,0.834576,...,1.061344,1.093455,1.098647,1.125250,1.113206,1.123011,1.098978,1.101400,1.067910,1.056974


In [7]:
dfCoAgg.to_csv('WkDay-WkEnd-Factors-ByCounty-ByMonth.csv', index=False)